# Advanced Learning Algorithms

## Import Modules

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.optimizers import Adam

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor

## Import Data

We're using data from the Titanic Competition. 

In [9]:
data = pd.read_csv("./titanic/train.csv")
test = pd.read_csv("./titanic/test.csv")

## Main Content

### Neural Network, Neural Network Training

### Advice for Applying Machine Learning

### Decision Tree